In [1]:
import glob
import esmlab
import xarray as xr
from utils import copy_fill_settings_ds, time_set_mid

In [2]:
base_dir = '/glade/collections/cdg/data/CMIP6/CMIP'
institute = 'NCAR'
model = 'CESM2'
experiment = 'historical'
rip = 'r7i1p1f1'
MIP_table = 'Omon'
varname = 'fgco2'
grid = 'gn'

dir_in = '/'.join([base_dir, institute, model, experiment, rip, MIP_table, varname, grid, 'latest'])
fname_pattern = '_'.join([varname, MIP_table, model, experiment, rip, grid, '*.nc'])
fnames = glob.glob(dir_in+'/'+fname_pattern)
fnames.sort()
print(fnames)

['/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_185001-189912.nc', '/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_190001-194912.nc', '/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_195001-199912.nc', '/glade/collections/cdg/data/CMIP6/CMIP/NCAR/CESM2/historical/r7i1p1f1/Omon/fgco2/gn/latest/fgco2_Omon_CESM2_historical_r7i1p1f1_gn_200001-201412.nc']


In [3]:
dir_out = '/glade/scratch/klindsay/CESM2_CMIP6_ocnBgchem'
yr_lo = '1999'
yr_hi = '2000'

yr_fname_string = yr_lo + '-' + yr_hi

fname_out = '_'.join([varname, MIP_table, model, experiment, rip, grid, yr_fname_string, 'mon_clim.nc'])
print(fname_out)

fgco2_Omon_CESM2_historical_r7i1p1f1_gn_1999-2000_mon_clim.nc


In [4]:
time_name = 'time'

# save time encoding from first file, to restore it in the multi-file case
with xr.open_dataset(fnames[0]) as ds_in:
    time_encoding = ds_in[time_name].encoding

/glade/work/klindsay/miniconda3/envs/CESM2_CMIP6_ocnBgchem/lib/python3.7/site-packages/xarray/conventions.py:494: SerializationWarning: variable 'fgco2' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  use_cftime=use_cftime,


In [5]:
# data_vars = 'minimal', to avoid introducing time dimension
# to time-invariant fields when there are multiple files
ds_in = xr.open_mfdataset(fnames, data_vars='minimal', combine='by_coords')

# restore time encoding, needed in multi-file case
ds_in[time_name].encoding = time_encoding

# missing_value attribute CESM2 CMIP6 output is wrong type
ds_in[varname].encoding['missing_value'] = ds_in[varname].encoding['_FillValue']

# reset time values to mid-interval, to ensure climatology works correctly
time_set_mid(ds_in, time_name)

yyyymmdd_lo = yr_lo + '-01-01'
yyyymmdd_hi = yr_hi + '-12-31'

ds_clim = esmlab.climatology(ds_in.sel(time=slice(yyyymmdd_lo, yyyymmdd_hi)), freq='mon')

# set essential time encoding values
# as esmlab.climatology only sets dtype and _FillValue
ds_clim[time_name].encoding['units'] = time_encoding['units']
ds_clim[time_name].encoding['calendar'] = time_encoding['calendar']

# get rid of NaN _FillValue's
copy_fill_settings_ds(ds_in, ds_clim)

# document input files
ds_clim.attrs['input_file_list'] = ' '.join(fnames)

ds_clim.to_netcdf('/'.join([dir_out, fname_out]))

/glade/work/klindsay/miniconda3/envs/CESM2_CMIP6_ocnBgchem/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
